In [1]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cgokmen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Complained object azqqwx does not exist in the database. Skipping.
Complained object ntmrkt does not exist in the database. Skipping.
Complained object aginyz does not exist in the database. Skipping.
Complained object bjyvil does not exist in the database. Skipping.
Complained object yzplpe does not exist in the database. Skipping.
Complained object asjalm does not exist in the database. Skipping.


In [2]:
tasks = Task.view_challenge()
for task in tasks:
    # print(task.name, [s.name for s in task.matched_scenes])
    if not task.matched_scenes:
        print(task.name)
    if task.uses_cloth:
        print(task.name, "uses cloth", {x for x in task.synsets if "cloth" in x.property_names})
print(len(tasks), "tasks")

setting_the_fire-0
wash_dog_toys-0 uses cloth {Synset(name='rag.n.01')}
wash_goalkeeper_gloves-0 uses cloth {Synset(name='hand_towel.n.01')}
104 tasks


In [3]:
# Starting pick
ig_scenes = {x for x in Scene.all_objects() if x.name.endswith("_int")}
starting_set = {Scene.get(x) for x in {   # ig_scenes | {
    "house_single_floor",
    "house_double_floor_lower",
    "Beechwood_0_garden",
    "Rs_garden",
}}

In [10]:
import random

OFFSETS = {Scene.get(k): v for k, v in {
    "house_single_floor": -30,  # encourage hsf
    "house_double_floor_upper": -100,  # encourage upper
    "Beechwood_0_garden": 5,  # discourage beechwood
    "Rs_garden": 5,  # discourage beechwood
}.items()}
def assign_activities_balanced(scene_set, dont_assign=None):
    # 2. Initialize assignment and location loads
    assignment = {}
    location_load = {scene: 0 for scene in scene_set}
    location_load[None] = 0  # For unassigned tasks

    # Add a negative count to house single floor to increase its count
    for sc, offset in OFFSETS.items():
        if sc not in location_load:
            continue
        location_load[sc] += offset

    # 3. Shuffle activities to avoid bias from input order (optional but recommended)
    task_set = list(tasks)
    random.shuffle(task_set)

    # 4. Greedy assignment process
    for task in task_set:
        possible_locations = set(task.matched_scenes) & scene_set

        if dont_assign and task in dont_assign:
            possible_locations = possible_locations - {dont_assign[task]}
        
        possible_locations = list(possible_locations)

        if not possible_locations:
            assignment[task] = None
            location_load[None] += 1
            continue

        # Find minimum load and randomly choose among ties
        min_load = min(location_load[loc] for loc in possible_locations)
        best_options = [loc for loc in possible_locations if location_load[loc] == min_load]
        chosen_location = random.choice(best_options)

        # Assign and update load
        assignment[task] = chosen_location
        location_load[chosen_location] += 1

    for sc, offset in OFFSETS.items():
        if sc not in location_load:
            continue
        location_load[sc] -= offset

    total_load = sum(location_load.values())
    assert total_load == len(tasks), "Total load mismatch: expected {}, got {}".format(len(tasks), total_load)
    return assignment, location_load

In [11]:
# What activities do these scenes cover?
def cover(scene_set):
    return {task for task in tasks if set(task.matched_scenes) & scene_set}

print(len(cover(starting_set)))
assignment, counts = assign_activities_balanced(starting_set)
print(counts)

95
{Scene(name='Rs_garden'): 14, Scene(name='house_single_floor'): 49, Scene(name='house_double_floor_lower'): 18, Scene(name='Beechwood_0_garden'): 14, None: 9}


In [14]:
for t, s in assignment.items():
    if s is not None:
        continue
    print(f"https://behavior.stanford.edu/knowledgebase/tasks/{t}/index.html")

https://behavior.stanford.edu/knowledgebase/tasks/taking_fish_out_of_freezer-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/carrying_in_groceries-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/turning_off_the_hot_tub-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/buying_fast_food-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/bag_groceries-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/watering_outdoor_flowers-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/hanging_pictures-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/setting_the_fire-0/index.html
https://behavior.stanford.edu/knowledgebase/tasks/turn_off_a_normal_school_calculator-0/index.html


In [13]:
# Do a second round to assign a second scene to each task. Don't use the first scene assigned to each task.
assignment2, counts2 = assign_activities_balanced(starting_set, dont_assign=assignment)
print(counts2)

{Scene(name='Rs_garden'): 14, Scene(name='house_single_floor'): 34, Scene(name='house_double_floor_lower'): 18, Scene(name='Beechwood_0_garden'): 18, None: 20}


In [ ]:
# Greedy algorithm step
def next_pick(scene_set):
    candidates = set(Scene.all_objects()) - scene_set
    current_cover = cover(scene_set)
    advantages = {x: cover(scene_set | {x}) - current_cover for x in candidates}
    return dict(sorted(advantages.items(), key=lambda x: -len(x[1])))
{k: len(v) for k, v in next_pick(starting_set).items()}

In [ ]:
# Run the greedy algorithm
max_cover = len(cover(scenes))
greedy_selection = set(starting_set)
while len(cover(greedy_selection)) < max_cover:
    pick = next_pick(greedy_selection)
    print("Adding", pick)
    greedy_selection.add(pick[0])
print("Final set:", sorted(greedy_selection))
print(len(greedy_selection))

In [ ]:
import itertools, tqdm

print("Max possible cover:", max_cover)

# Run a non-greedy version
def combinatorial_search():
    max_extra_scenes_needed = len(greedy_selection) - len(starting_set)
    print("We will process up to", max_extra_scenes_needed, "extra scenes")
    extra_scene_candidates = scenes - starting_set
    print("We have", len(extra_scene_candidates), "candidates")
    
    def combinatorial_search_n(extra_to_use):
        best_cover = 0
        best_cover_set = set()
        
        print("Processing", extra_to_use, "element combinations")
        for combination_extra in tqdm.tqdm(itertools.combinations(extra_scene_candidates, extra_to_use)):
            combination = starting_set | set(combination_extra)
            combination_cover = len(cover(combination))

            if combination_cover >= best_cover:
                best_cover = combination_cover
                best_cover_set = combination
            
            if combination_cover == max_cover:
                break
                
        return best_cover_set, best_cover
    
    for extra_to_use in range(1, max_extra_scenes_needed + 1):
        combination, combination_cover = combinatorial_search_n(extra_to_use)
        print(f"Best total with {extra_to_use} elements: {combination_cover}. Scenes: {sorted(combination - starting_set)}")
        
        if combination_cover == max_cover:
            print("Final set.")
            return
            
combinatorial_search()

In [ ]:
# Pomaria-advantage
print(len(cover(starting_set | {"Pomaria_0_garden"})) - len(cover(starting_set)))

In [ ]:
final_set = starting_set | {'Pomaria_0_garden', 'grocery_store_cafe', 'office_large', 'restaurant_brunch'}
len(cover(final_set))

In [ ]:
print(final_set - ig_scenes)